In [ ]:
#Mount Drive
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


## Imports

In [ ]:
import os
import pandas as pd

In [ ]:
KAGGLE_PERMISSION_PATH='/content/drive/MyDrive/Owais/Code/kaggle.json'

## Load Dataset

In [ ]:
# Install the Kaggle library
!pip install kaggle --q

In [ ]:
# Create a directory named ".kaggle"
!mkdir ~/.kaggle

In [ ]:
# Copy the "kaggle.json" file into this new directory
!cp /content/drive/MyDrive/Owais/Code/kaggle.json ~/.kaggle/kaggle.json

# Change the permissions of the "kaggle.json" file
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d tpotterer/motley-fool-scraped-earnings-call-transcripts

 94% 253M/269M [00:01<00:00, 209MB/s]
100% 269M/269M [00:01<00:00, 181MB/s]


In [ ]:
!unzip motley-fool-scraped-earnings-call-transcripts.zip

Archive:  motley-fool-scraped-earnings-call-transcripts.zip
  inflating: motley-fool-data.pkl    


## Read Data

In [ ]:
df= pd.read_pickle('/content/motley-fool-scraped-earnings-call-transcripts.zip')
df.head(5)

,date,exchange,q,ticker,transcript
0,"Aug 27, 2020, 9:00 p.m. ET",NASDAQ: BILI,2020-Q2,BILI,"Prepared Remarks:\nOperator\nGood day, and wel..."
1,"Jul 30, 2020, 4:30 p.m. ET",NYSE: GFF,2020-Q3,GFF,Prepared Remarks:\nOperator\nThank you for sta...
2,"Oct 23, 2019, 5:00 p.m. ET",NASDAQ: LRCX,2020-Q1,LRCX,Prepared Remarks:\nOperator\nGood day and welc...
3,"Nov 6, 2019, 12:00 p.m. ET",NASDAQ: BBSI,2019-Q3,BBSI,"Prepared Remarks:\nOperator\nGood day, everyon..."
4,"Aug 7, 2019, 8:30 a.m. ET",NASDAQ: CSTE,2019-Q2,CSTE,Prepared Remarks:\nOperator\nGreetings and wel...


## Preprocessing

In [ ]:
!pip install contractions --q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 6.4 MB/s eta 0:00:00


In [ ]:
df['quarter'] = df['q'].str.extract(r'Q(\d+)')

In [ ]:
from dateutil import parser

def convert_to_timestamp(date_str):
    try:
        return parser.parse(date_str)
    except:
        return None

#df['date_processed'] = df['date'].apply(convert_to_timestamp)
df['date_only'] = df['date'].apply(convert_to_timestamp).dt.date


# Create a new DataFrame containing rows with None values in 'date_column'
filtered_df = df[df['date_only'].isnull()]

/usr/local/lib/python3.10/dist-packages/dateutil/parser/_parser.py:1207: UnknownTimezoneWarning: tzname ET identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "


In [ ]:
#add column that includes the total character count for each earning call transcripts
df['char_count']= df['transcript'].str.len()
df[['transcript', 'char_count']].head()

,transcript,char_count
0,"Prepared Remarks:\nOperator\nGood day, and wel...",35736
1,Prepared Remarks:\nOperator\nThank you for sta...,29761
2,Prepared Remarks:\nOperator\nGood day and welc...,53094
3,"Prepared Remarks:\nOperator\nGood day, everyon...",47532
4,Prepared Remarks:\nOperator\nGreetings and wel...,27273


In [ ]:
import contractions
import re
import nltk
import string
nltk.download('punkt')
from nltk import word_tokenize

def preprocess_text(text):
    if isinstance(text, str):

        text= re.sub(r'((https?):((//)|(\\\\))+([\w\d:#@%/;$()~_?\+-=\\\.&](#!)?)*)', ', ', text)

        text= ' '.join([word.strip() for word in text.split() if word.strip() and word[0] not in ['@', '#', '']])

        #remove quotes and apostrophes
        text=re.sub('[^\w\s]','',text)

        # clean up contractions
        text = contractions.fix(text)

        # lowercase text
        text = text.lower()

        # strip html from the text
        text = re.sub('<[^<]+?>', '', text)

        # strip out standalone digits
        text = re.sub(r'\b\d+\b', '', text)

        # tokenize text into individual terms
        tokens = word_tokenize(text)

        # pass each token through the function that determines if we keep it or not.
        clean_tokens = list(filter(None, tokens))
        return clean_tokens

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
df['preprocess_trans']= df['transcript'].apply(lambda x : ' '.join(preprocess_text(x)))

In [ ]:
df['preprocess_trans'].sample(20)

5743     prepared remarks operator greetings welcome to...
15090    prepared remarks operator thank you for standi...
2440     prepared remarks operator welcome to the xylem...
5112     prepared remarks operator ladies and gentlemen...
6112     prepared remarks operator good day and thank y...
2378     prepared remarks operator good morning my name...
17620    prepared remarks operator hello and thank you ...
3908     prepared remarks operator greetings and welcom...
4288     prepared remarks operator good day everyone an...
18665    prepared remarks operator ladies and gentlemen...
12931    prepared remarks operator good day and thank y...
9395     prepared remarks operator good morning and wel...
12921    prepared remarks operator good afternoon ladie...
10721    prepared remarks operator ladies and gentlemen...
10781    prepared remarks operator good afternoon my na...
994      prepared remarks operator good day ladies and ...
10873    prepared remarks operator good morning and wel.

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18755 entries, 0 to 18754
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   date        18755 non-null  object
 1   exchange    18755 non-null  object
 2   q           18755 non-null  object
 3   ticker      18755 non-null  object
 4   transcript  18755 non-null  object
 5   date_only   18375 non-null  object
dtypes: object(6)
memory usage: 879.3+ KB


## Topics Tags

In [ ]:
df=df_full.copy()

In [ ]:
!pip install transformers --q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 47.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 50.3 MB/s eta 0:00:00


In [ ]:
df['transcript'][0]

In [ ]:
from transformers import BertTokenizer

# Load the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Custom lists of words/phrases for financial metrics
revenue_keywords = [
    "sales", "revenue", "top line", "top bottom line",
    "net revenue", "organic revenue growth",
    "organic sales growth", "operational sales",
    "income", "gross income", "net sales", "sales growth",
    "revenue growth", "sales revenue", "sales report",
    "revenue report", "revenue generation", "turnover",
    "sales turnover", "total revenue", "earnings report",
    "sales performance", "revenue performance"

]

expenses_keywords = ["expenses", "costs", "expenditures", "outlays",
    "overheads", "outgoings", "disbursements", "payments",
    "operating costs", "administrative expenses", "variable costs",
    "fixed costs", "capital expenditures", "operating expenses",
    "indirect costs", "direct costs", "overhead costs", "cost of goods sold",
    "budgets", "investments", "spending"]

profit_keywords = [
    "profit", "net income", "earnings", "gain",
    "margin", "gross margin", "operating margin",
    "return invested capital", "return capital",
    "net profit", "operating profit", "bottom line",
    "net earnings", "revenue surplus", "net gain",
    "profitability", "profitability ratio", "profitability index",
    "earnings report", "profitability analysis",
    "net profit margin", "earnings growth",
    "return on investment", "ROI", "earnings per share",
    "EPS", "net operating income",
    "profit before tax", "PBT", "net profit after tax", "NPAT"
]

# Earnings call transcript
transcript = df['transcript'][1]
# Tokenize transcript
tokenized_transcript = tokenizer.tokenize(transcript)

# Placeholder for detected entities
detected_entities = {"Revenue": [], "Expenses": [], "Profit": []}

# Iterate through tokens and match with custom lists
for token in tokenized_transcript:
    if token in revenue_keywords:
        detected_entities["Revenue"].append(token)
    elif token in expenses_keywords:
        detected_entities["Expenses"].append(token)
    elif token in profit_keywords:
        detected_entities["Profit"].append(token)

# Print detected entities
print("Detected Entities:")
for entity_type, entities in detected_entities.items():
    print(f"Type: {entity_type}\t Entities: {', '.join(entities)}")


Detected Entities:
Type: Revenue	 Entities: revenue, sales, revenue, sales, sales, sales, revenue, revenue, income, income, revenue, revenue, revenue, revenue, revenue, revenue, revenue, sales, sales, sales, sales
Type: Expenses	 Entities: expenses, spending, costs, costs, costs, expenses, expenses, expenses, expenditures, expenses, spending
Type: Profit	 Entities: earnings, earnings, earnings, profit, margin, margin, margin, earnings, margin, margin, gain, earnings


## Get Stock Price

In [ ]:
import pandas as pd
import re

def preprocess_ticker(ticker):
    cleaned_ticker = re.sub(r'[^A-Za-z0-9]', '', ticker)
    return cleaned_ticker

# Preprocess the ticker column
#df['cleaned_ticker'] = df['ticker'].apply(preprocess_ticker)

In [ ]:
# Mapping of tickers to be replaced
mapping = {
    'AX.DL': 'AX-DL',
    'BF.B': 'BF-B',
    'BIO.B': 'BIO-B',
    'DISC.A': 'DISC-A',
    'HCRS.Q': 'HCRS-Q',
    'JW.A': 'JW-A',
    'LBRD.A': 'LBRD-A',
    'MOG.A': 'MOG-A',
    'RDS.A': 'RDS-A',
    'ZLND.Y': 'ZNGA',
}

# Replace values in 'ticker' column using the mapping
sp_none['ticker'] = sp_none['ticker'].replace(mapping)


### Get the price in next quarter

In [ ]:
sp_df=data=pd.read_csv('/content/drive/MyDrive/Owais/stockprices.csv')

In [ ]:
sp_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18375 entries, 0 to 18374
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   date         18375 non-null  object 
 1   exchange     18375 non-null  object 
 2   q            18375 non-null  object 
 3   ticker       18375 non-null  object 
 4   transcript   18375 non-null  object 
 5   date_only    18375 non-null  object 
 6   stock_price  16935 non-null  float64
dtypes: float64(1), object(6)
memory usage: 1005.0+ KB


In [ ]:
# Trial on one ticker to Calculate the next quarter
''' call_date is "2020-8-27" '''
next_quarter = data['date_only'][0] + pd.DateOffset(months=3)
print("Next Quarter:", next_quarter)

Next Quarter: 2020-11-27 00:00:00


In [ ]:
import yfinance as yf
import datetime
from datetime import timedelta

sp_df['stock_price_after_call'] = None

for index, row in sp_df.iterrows():
    ticker = row['ticker']
    start_date=datetime.datetime.strptime(row['date_only'], "%Y-%m-%d")+pd.DateOffset(months=3)
    #start_date = row['date_only'] + pd.DateOffset(months=3) # Use the 'date_only' value as start date
    end_date = start_date + timedelta(days=1)  # Calculate end date as one day after start date

    try:
        # Download the stock price
        stock = yf.download(ticker.strip(), start=start_date, end=end_date, auto_adjust=True)

        if len(stock) > 0:
            # Extract the 'Close' price and update the 'stock_price' column in the DataFrame
            close_price = stock['Close'].iloc[0]
            sp_df.at[index, 'stock_price_after_call'] = close_price

    except Exception as e:
        print(f"Error downloading {ticker} data:", e)


Output hidden; open in https://colab.research.google.com to view.

In [ ]:
sp_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18375 entries, 0 to 18374
Data columns (total 8 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   date                    18375 non-null  object 
 1   exchange                18375 non-null  object 
 2   q                       18375 non-null  object 
 3   ticker                  18375 non-null  object 
 4   transcript              18375 non-null  object 
 5   date_only               18375 non-null  object 
 6   stock_price             16935 non-null  float64
 7   stock_price_after_call  14314 non-null  object 
dtypes: float64(1), object(7)
memory usage: 1.1+ MB


In [ ]:
sp_df

,date,exchange,q,ticker,transcript,date_only,stock_price,stock_price_after_call
0,"Aug 27, 2020, 9:00 p.m. ET",NASDAQ: BILI,2020-Q2,BILI,"Prepared Remarks:\nOperator\nGood day, and wel...",2020-08-27,45.139999,64.860001
1,"Jul 30, 2020, 4:30 p.m. ET",NYSE: GFF,2020-Q3,GFF,Prepared Remarks:\nOperator\nThank you for sta...,2020-07-30,17.238564,17.968714
2,"Oct 23, 2019, 5:00 p.m. ET",NASDAQ: LRCX,2020-Q1,LRCX,Prepared Remarks:\nOperator\nGood day and welc...,2019-10-23,221.749268,298.38504
3,"Nov 6, 2019, 12:00 p.m. ET",NASDAQ: BBSI,2019-Q3,BBSI,"Prepared Remarks:\nOperator\nGood day, everyon...",2019-11-06,87.694138,80.516602
4,"Aug 7, 2019, 8:30 a.m. ET",NASDAQ: CSTE,2019-Q2,CSTE,Prepared Remarks:\nOperator\nGreetings and wel...,2019-08-07,14.448421,14.972457
...,...,...,...,...,...,...,...,...
18370,"Nov 9, 2021, 1:00 p.m. ET",NYSE: SWX,2021-Q3,SWX,Prepared Remarks:\nOperator\nLadies and gentle...,2021-11-09,66.814957,66.622871
18371,"Nov 18, 2021, 12:00 p.m. ET",NYSE: PNNT,2021-Q4,PNNT,"Prepared Remarks:\nOperator\nGood morning, and...",2021-11-18,7.020000,7.63
18372,"Feb 08, 2022, 11:00 a.m. ET",NYSE: TDG,2022-Q1,TDG,Prepared Remarks:\nOperator\nThank you for sta...,2022-02-08,645.820007,None
18373,"Feb 28, 2022, 4:30 p.m. ET",NASDAQ: DVAX,2021-Q4,DVAX,"Prepared Remarks:\nOperator\nGood day, ladies ...",2022-02-28,12.260000,None


### Get the price in the day of call

In [ ]:
import yfinance as yf
import datetime
from datetime import timedelta

data=df.dropna(subset=['date_only'])
data['stock_price'] = None

for index, row in data.iterrows():
    ticker = row['ticker']
    start_date = row['date_only']  # Use the 'date_only' value as start date
    end_date = start_date + timedelta(days=1)  # Calculate end date as one day after start date

    try:
        # Download the stock price
        stock = yf.download(ticker.strip(), start=start_date, end=end_date, auto_adjust=True)

        if len(stock) > 0:
            # Extract the 'Close' price and update the 'stock_price' column in the DataFrame
            close_price = stock['Close'].iloc[0]
            data.at[index, 'stock_price'] = close_price

    except Exception as e:
        print(f"Error downloading {ticker} data:", e)


[*********************100%%**********************]  1 of 1 completed

<ipython-input-13-dcaa516bbde3>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['stock_price'] = None



[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['WDR']: Exception('%ticker%: No timezone found, symbol may be delisted')



[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

...

ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['HZN']: Exception('%ticker%: No timezone found, symbol may be delisted')



[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%******

ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['IEA']: Exception('%ticker%: No timezone found, symbol may be delisted')



[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['CIB)']: Exception('%ticker%: No timezone found, symbol may be delisted')
